# Dynamic Portfolios

and intertemporal hedging in a simple discrete time model.

## Load Packages

In [1]:
using Dates, LinearAlgebra, Distributions

include("jlFiles/printmat.jl")
include("jlFiles/printTable.jl")

printTable2

# Log Utility

We first find the *myopic* (period by period) optimal portfolio when the investor has log utility,

$
U(W) = \ln (1+R_p),
$

where $R_p$ is the portfolio return.

In [2]:
rf   = 0.03          #riskfree rate             
σ²   = 100/10000     #variance of risky asset
μe1A = 1/100         #mean excess return of single risky asset, state A
μe1B = 0.5/100       #state B

vA = (μe1A + σ²/2)/σ²
vB = (μe1B + σ²/2)/σ²

rowNames1 = ["risky asset","riskfree"]
printblue("Portfolio weights in the two states (the case of a single risky asset):")
printTable([vA vB;1-sum(vA) 1-sum(vB)],["state A","state B"],rowNames1)

Portfolio weights in the two states (the case of a single risky asset):
              state A   state B
risky asset     1.500     1.000
riskfree       -0.500     0.000



In [3]:
"""
Calculate optimal portfolio for log utility case, when the log returns are N(μe+rf,Σ),
Campbell&Viceira. 
"""
function OptPortLogUtil(Σ,μe,rf)     
  v     = inv(Σ)*(μe+diag(Σ)/2)      
  Erp   = rf + v'μe + v'diag(Σ)/2 - v'Σ*v/2
  Varrp = v'Σ*v
  return v, Erp, Varrp
end

OptPortLogUtil

In [4]:
Σ  = [166  34  58;              #3 risky assets
       34  64   4;
       58   4 100]/10000
μeA = [2.0, 1.0, 0.5]/100       #state A
μeB = [2.0, 0.0, 0.5]/100       #state B

println()

vA, = OptPortLogUtil(Σ,μeA,rf)  #myopic portfolio choice in each state
vB, = OptPortLogUtil(Σ,μeB,rf)

printblue("Portfolio weights in the two states (several risky assets):")
rowNames3 = ["asset 1","asset 2","asset 3","riskfree"]
printTable([vA vB;1-sum(vA) 1-sum(vB)],["state A","state B"],rowNames3)


Portfolio weights in the two states (several risky assets):
           state A   state B
asset 1      1.384     1.810
asset 2      1.318    -0.460
asset 3      0.144    -0.031
riskfree    -1.847    -0.319



# CRRA Utility

We now turn to the *myopic* (period by period) optimal portfolio when the investor has CRRA utility

$
U(W) = (1+R_p)^{1-\gamma}/(1-\gamma),
$

where $R_p$ is the portfolio return. As $\gamma \rightarrow 1$, this becomes the same as log utility (need to take the limit to prove that).

In [5]:
γ = 3                               #risk aversion 

vA = (μe1A + σ²/2)/(σ²*γ)           #with a single risky asset
vB = (μe1B + σ²/2)/(σ²*γ)

printblue("Portfolio weights in the two states (the case of a single risky asset):")
printTable([vA vB;1-sum(vA) 1-sum(vB)],["state A","state B"],rowNames1)

Portfolio weights in the two states (the case of a single risky asset):
              state A   state B
risky asset     0.500     0.333
riskfree        0.500     0.667



In [6]:
"""
Calculate optimal portfolio for CRRA utility case, when the log returns are N(μe+rf,Σ),
Campbell&Viceira. 
"""
function OptPortCRRA(Σ,μe,rf,γ)       
  v     = inv(Σ)*(μe+diag(Σ)/2)/γ     
  Erp   = rf + v'μe + v'diag(Σ)/2 - v'Σ*v/2
  Varrp = v'Σ*v
  return v, Erp, Varrp
end

OptPortCRRA

In [7]:
γ = 3

vA, = OptPortCRRA(Σ,μeA,rf,γ)
vB, = OptPortCRRA(Σ,μeB,rf,γ)

printblue("Portfolio weights in the two states (several risky assets):")
printTable([vA vB;1-sum(vA) 1-sum(vB)],["state A","state B"],rowNames3)

Portfolio weights in the two states (several risky assets):
           state A   state B
asset 1      0.461     0.603
asset 2      0.439    -0.153
asset 3      0.048    -0.010
riskfree     0.051     0.560



# Intertemporal Hedging

We now consider the more difficult case when the CRRA investor considers several periods. 

In the case of log utility ($\gamma=1$), this actually gives a myopic solution (in each period, $\ln(1+R_p$) is maximized, where $R_p$ is the one period portfolio return.

With CRRA this may no longer hold. In particular, if there are some predicatable (non-iid) features of the asset returns, then today's investment may be influenced by how the return over the next period correlated with the investment opportunities in the subsequent periods.

The optimal solution (see lecture notes) is for the case when the vector ($n$ assets) of excess returns follow

$
r_{t+1}^{e}=a+z_{t}+u_{t+1},
$

where the vector $z_t$ follows the VAR(1)

$
z_{t+1}=\phi z_{t}+\eta_{t+1},
$
with $\eta_{t+1}$ being $N(\mathbf{0},\Sigma_{\eta})$.

The covariance matrix of $u_{t+1}$ and $\eta_{t+1}$, which plays a key role of the analysis, is denoted by $\Sigma_{u\eta}$.

## A Single Risky Asset 

and a riskfree

In [8]:
"""
Optimal dynamic portfolio choice when there is a single risky asset. 
"""
function CRRAPortOpt1(Σ_u,Σ_η,Σ_uη,a,ϕ,γ,z)
  v_myop  = inv(Σ_u)*(a + z + Σ_u/2)/γ
  Σ       = 2*Σ_u + Σ_η + 2*Σ_uη
  v_noreb = inv(Σ)*(2*a + (I+ϕ)*z + Σ/2)/γ
  Ev1     = inv(Σ_u)*(a + ϕ*z + Σ_u/2)/γ
  v_rebal = inv(Σ_u)*(a + z + Σ_u/2 + (1-γ)*Σ_uη*Ev1)/γ
  return v_myop, v_noreb, v_rebal
end

CRRAPortOpt1

In [9]:
(γ,Σ_u,Σ_η,a,ϕ) = (3,100/10000,75/10000,0.75/100,0.9)     #parameters in example with 1 risky asset

Σ_uηL  = -0.5*sqrt(Σ_u*Σ_η)     #corr(u,η)=-0.5
(v_myopL,v_norebL,v_rebalL) = CRRAPortOpt1(Σ_u,Σ_η,Σ_uηL,a,ϕ,γ,0)

printblue("Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η)=-0.5")
xx = [v_myopL   v_rebalL;
      1-sum(v_myopL) 1-sum(v_rebalL)]
printTable(xx,["myopic","2-period investor"],rowNames1,width=20)

Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η)=-0.5
                         myopic   2-period investor
risky asset               0.417               0.537
riskfree                  0.583               0.463



In [10]:
Σ_uηH  =  0.5*sqrt(Σ_u*Σ_η)     #corr(u,η)=0.5
(v_myopH,v_norebH,v_rebalH) = CRRAPortOpt1(Σ_u,Σ_η,Σ_uηH,a,ϕ,γ,0)

printblue("Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η)=0.5")
xx = [v_myopH   v_rebalH;
      1-sum(v_myopH) 1-sum(v_rebalH)]
printTable(xx,["myopic","2-period investor"],rowNames1,width=20)

Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η)=0.5
                         myopic   2-period investor
risky asset               0.417               0.296
riskfree                  0.583               0.704



## Three Risky Assets

and a riskfree

In [11]:
"""
Optimal dynamic portfolio choice when there are several risky assets 
"""
function CRRAPortOpt(Σ_u,Σ_η,Σ_uη,a,ϕ,γ,z)
  v_myop  = inv(Σ_u)*(a + z + diag(Σ_u)/2)/γ
  Σ       = 2*Σ_u + Σ_η + 2*Σ_uη
  v_noreb = inv(Σ)*(2*a + (I+ϕ)*z + diag(Σ)/2)/γ
  Ev1     = inv(Σ_u)*(a + ϕ*z + diag(Σ_u)/2)/γ
  v_rebal = inv(Σ_u)*(a + z + diag(Σ_u)/2 + (1-γ)*Σ_uη*Ev1)/γ
  return v_myop, v_noreb, v_rebal
end

CRRAPortOpt

In [12]:
γ = 3

Σ_u    = [166  34  58;
           34  64   4;
           58   4 100]/10000
Σ_η    = [0  0   0;
          0  100 0;
          0  0   0]/10000       #only asset 2 has dynamics in expected returns
a      = [2,0.5,0.5]/100
ϕ      = [0  0   0;             #doesn't matter for 2-period problem when z_t=0
          0  0.9 0;
          0  0   0]

Σ_uηL =  [0  0   0;            #Cov(u,η)<0 for asset 2
          0  -40  0;
          0  0    0]/10000       
Σ_uηH =  [0  0   0;            #Cov(u,η)>0 for asset 2
          0  40   0;
          0  0    0]/10000       

printblue("The model parameters:\n")

println("Σ_u*10000")
printmat(Σ_u*10000)

println("Σ_η*10000")
printmat(Σ_η*10000)

println("a*100")
printmat(a*100)

println("\nTwo cases: negative and positive Cov(u,η) for asset 2")
printmat(Σ_uηL*10000)
printmat(Σ_uηH*10000)

The model parameters:

Σ_u*10000
   166.000    34.000    58.000
    34.000    64.000     4.000
    58.000     4.000   100.000

Σ_η*10000
     0.000     0.000     0.000
     0.000   100.000     0.000
     0.000     0.000     0.000

a*100
     2.000
     0.500
     0.500


Two cases: negative and positive Cov(u,η) for asset 2
     0.000     0.000     0.000
     0.000   -40.000     0.000
     0.000     0.000     0.000

     0.000     0.000     0.000
     0.000    40.000     0.000
     0.000     0.000     0.000



In [13]:
(v_myopL,v_norebL,v_rebalL) = CRRAPortOpt(Σ_u,Σ_η,Σ_uηL,a,ϕ,γ,zeros(3))     #different Cov(u,η)
(v_myopH,v_norebH,v_rebalH) = CRRAPortOpt(Σ_u,Σ_η,Σ_uηH,a,ϕ,γ,zeros(3))


printblue("Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η) < 0")
xx = [v_myopL   v_rebalL;
      1-sum(v_myopL) 1-sum(v_rebalL)]
printTable(xx,["myopic","2-period investor"],rowNames3,width=20)

printblue("Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η) > 0")
xx = [v_myopH   v_rebalH;
      1-sum(v_myopH) 1-sum(v_rebalH)]
printTable(xx,["myopic","2-period investor"],rowNames3,width=20)

Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η) < 0
                      myopic   2-period investor
asset 1                0.532               0.516
asset 2                0.143               0.211
asset 3                0.019               0.026
riskfree               0.306               0.247

Portfolio weights: myopic and 2-period investor (who can rebalance), corr(u,η) > 0
                      myopic   2-period investor
asset 1                0.532               0.549
asset 2                0.143               0.075
asset 3                0.019               0.012
riskfree               0.306               0.364

